In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib
# 파일 경로
승하차_파일 = "../../data/결과/승하차/통합/1호선_승하차인원_통합.csv"
혼잡도_파일 = "../../data/결과/혼잡도/통합/1호선_혼잡도_통합.csv"

In [11]:
# 정원
정원 = 2000

# 데이터 불러오기
df_flow_raw = pd.read_csv(승하차_파일, encoding="euc-kr")
df_crowd_raw = pd.read_csv(혼잡도_파일, encoding="euc-kr")

In [12]:
# 시계열 시간 컬럼들 (5:00 ~ 0:00)
time_cols = [
    "5:00",
    "6:00",
    "7:00",
    "8:00",
    "9:00",
    "10:00",
    "11:00",
    "12:00",
    "13:00",
    "14:00",
    "15:00",
    "16:00",
    "17:00",
    "18:00",
    "19:00",
    "20:00",
    "21:00",
    "22:00",
    "23:00",
    "0:00",
]

# melt
df_flow = df_flow_raw.melt(
    id_vars=["역번호", "평일주말", "구분", "호선", "역명"],
    value_vars=time_cols,
    var_name="시간",
    value_name="승하차인원",
)

df_crowd = df_crowd_raw.melt(
    id_vars=["역번호", "평일주말", "구분", "호선", "역명"],
    value_vars=time_cols,
    var_name="시간",
    value_name="혼잡도",
)

In [13]:
# 공통으로 병합하기 위한 구분 값 정리
# 구분: "승차" → 1, "하차" → 0
df_flow["승차"] = df_flow["구분"].map({"승차": 1, "하차": 0})
df_crowd["승차"] = df_crowd["구분"].map({"승차": 1, "하차": 0})

# 시간: "5:00" → 5 (정수형)
df_flow["hour"] = df_flow["시간"].str.replace(":00", "").astype(int)
df_crowd["hour"] = df_crowd["시간"].str.replace(":00", "").astype(int)

# 상행/하행 정보가 없다면 일단 고정값 (ex: 상행=1)
df_flow["상행"] = 1
df_crowd["상행"] = 1

In [15]:
# 타입 체크
print(df_flow[merge_keys].dtypes)
print(df_crowd[merge_keys].dtypes)

역번호      int64
평일주말    object
승차       int64
hour     int64
상행       int64
dtype: object
역번호       int64
평일주말     object
승차      float64
hour      int64
상행        int64
dtype: object


In [14]:
# 병합 키 지정
merge_keys = ["역번호", "평일주말", "승차", "hour", "상행"]

# inner 병합
df = pd.merge(
    df_flow[merge_keys + ["승하차인원"]],
    df_crowd[merge_keys + ["혼잡도"]],
    on=merge_keys,
    how="inner",
)

# 확인
print("✅ 최종 데이터 크기:", df.shape)
df.sample(3)

✅ 최종 데이터 크기: (0, 7)


ValueError: a must be greater than 0 unless no samples are taken

In [ ]:
# 결측, 음수 제거
df = df.dropna()
df = df[df["승하차인원"] >= 0]
df = df[df["혼잡도"] >= 0]
df = df[df["혼잡도"] <= 200]

In [ ]:
X = df[["역번호", "hour", "평일주말", "상행", "승차"]].values
y = df[["승하차인원", "혼잡도"]].values

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib

model = Sequential(
    [
        Dense(64, input_dim=X_train.shape[1], activation="relu"),
        Dense(32, activation="relu"),
        Dense(2),  # 승하차 인원, 혼잡도 예측
    ]
)

model.compile(optimizer="adam", loss="mse")
model.fit(X_train_scaled, y_train, epochs=30, batch_size=32, validation_split=0.2)

In [ ]:
model.save("./congestion_boarding_model.keras")
joblib.dump(scaler, "./congestion_boarding_scaler.pkl")